# 数据分析
* _id：案例的唯一标识符。
* context：案例内容，由法院判决书中的事实描述部分提取，分为标题和句子列表。
* question：针对案件提出的问题，每个案件都有一个问题。
* answer：问题的回答，可能是片段、YES/NO或"unknown"。
* supporting_facts：支持回答的问题的依据，包含标题和句子编号。

# config

In [ ]:
import argparse
import os
import json
from os.path import join

# 处理输入参数，设置最大查询长度和文档长度，并根据指定名称更新检查点和预测路径
def process_arguments(args):
    args.checkpoint_path = join(args.checkpoint_path, args.name) # 更新检查点路径为"检查点基路径/项目名称"
    args.prediction_path = join(args.prediction_path, args.name) # 更新预测文件保存路径为"预测文件基路径/项目名称"
    args.max_query_len = 50  # 最大查询长度设定为50，用于裁判文书问题的最大长度
    args.max_doc_len = 512   # 最大文档长度设定为512，用于裁判文书内容的最大长度

# 保存运行时的配置参数到JSON文件
def save_settings(args):
    os.makedirs(args.checkpoint_path, exist_ok=True)  # 创建检查点目录（如果不存在）
    os.makedirs(args.prediction_path, exist_ok=True)  # 创建预测结果目录（如果不存在）
    json.dump(args.__dict__, open(join(args.checkpoint_path, "run_settings.json"), 'w')) # 将配置保存为JSON格式

# 设置项目配置参数
def set_config():
    parser = argparse.ArgumentParser()
    data_path = 'output' # 默认输出文件夹路径

    # 定义必需和可选的命令行参数
    parser.add_argument("--name", type=str, default='default') # 项目名称，默认为"default"
    parser.add_argument("--prediction_path", type=str, default=join(data_path, 'submissions')) # 预测结果路径，默认为"output/submissions"
    parser.add_argument("--checkpoint_path", type=str, default=join(data_path, 'checkpoints')) # 检查点路径，默认为"output/checkpoints"
    parser.add_argument("--data_dir", type=str, default='data') # 数据目录，默认为"data"

    parser.add_argument("--fp16", action='store_true') # 是否使用FP16精度训练

    parser.add_argument("--ckpt_id", type=int, default=0) # 检查点ID，默认为0
    parser.add_argument("--bert_model", type=str, default='bert-base-uncased',
                        help='Currently only support bert-base-uncased and bert-large-uncased') # 使用的BERT模型，默认为'bert-base-uncased'

    # 学习和日志参数
    parser.add_argument("--epochs", type=int, default=4) # 训练轮数，默认为4
    parser.add_argument("--qat_epochs", type=int, default=0) # 量化感知训练轮数（QAT），默认为0
    parser.add_argument("--batch_size", type=int, default=32) # 批大小，默认为32
    parser.add_argument("--max_bert_size", type=int, default=8) # BERT层的最大尺寸，默认为8
    parser.add_argument("--eval_batch_size", type=int, default=32) # 评估时的批大小，默认为32
    parser.add_argument("--lr", type=float, default=2e-4) # 学习率，默认为2e-4
    parser.add_argument('--decay', type=float, default=1.0) # 学习率衰减率，默认为1.0
    parser.add_argument('--early_stop_epoch', type=int, default=0) # 提前停止训练的轮数，默认为0
    parser.add_argument("--verbose_step", default=50, type=int) # 显示训练进度的间隔步数，默认每50步显示一次
    parser.add_argument("--gradient_accumulation_steps", default=1, type=int) # 梯度累积步数，默认为1
    parser.add_argument("--seed", default=0, type=int) # 随机种子，默认为0

    parser.add_argument('--q_update', action='store_true', help='Whether update query') # 是否更新查询
    parser.add_argument("--prediction_trans", action='store_true', help='transformer version prediction layer') # 是否使用Transformer版本的预测层
    parser.add_argument("--trans_drop", type=float, default=0.5) # Transformer层的dropout率，默认为0.5
    parser.add_argument("--trans_heads", type=int, default=3) # Transformer层的头数，默认为3

    parser.add_argument("--input_dim", type=int, default=768, help="bert-base=768, bert-large=1024") # 输入维度，默认为768

    parser.add_argument("--model_gpu", default='0', type=str, help="device to place model.") # 训练模型的GPU编号，默认为'0'
    parser.add_argument('--trained_weight',default=None) # 预训练权重的路径，默认无

    # 损失函数相关参数
    parser.add_argument("--type_lambda", type=float, default=1) # 类型损失的权重，默认为1
    parser.add_argument("--sp_lambda", type=float, default=5) # 支持性事实的损失权重，默认为5
    parser.add_argument("--sp_threshold", type=float, default=0.5) # 支持性事实的阈值，默认为0.5
    parser.add_argument('--label_type_num', default=4, type=int)# 回答类型数目，包括yes/no/unknown/span，共4种

    args = parser.parse_args()

    process_arguments(args) # 调用处理参数的函数
    save_settings(args) # 调用保存设置的函数

    return args

# data_process
这段代码主要包括两个部分：read_examples和convert_examples_to_features，它们共同完成了数据处理的任务，从原始的文本数据中提取出模型训练所需的格式化数据。
# read_examples 函数
* 功能: 该函数负责读取原始的数据文件，提取出每一个问题（Question）、相关文档（Document）、支持事实（Supporting Facts）、答案（Answer）等信息，并将它们封装成Example对象列表。这一步是数据预处理的第一阶段，主要目的是从复杂的原始数据中提取出对模型训练有用的结构化信息。
* 原理: 函数通过遍历原始数据中的每一个案例，提取案例的关键信息。对于每个案例，它会记录问题ID、问题类型、文档中的句子、问题文本、支持事实的句子ID等信息。同时，它还会处理答案的位置，将答案文本在文档中的位置转换为基于单词的起始和结束位置。

# convert_examples_to_features 函数
* 功能: 该函数负责将Example对象转换为模型训练所需的特征格式，生成InputFeatures对象列表。这一步是数据预处理的第二阶段，目的是将上一步得到的结构化信息转换为模型可以直接处理的数值型特征。
* 原理: 函数首先使用BERT Tokenizer对问题文本和文档文本进行分词，然后将文本转换为对应的词汇表索引（input IDs）。同时，它还会生成注意力掩码（input mask）和段落ID（segment IDs）等特征。对于答案位置和支持事实的句子位置，函数会根据分词结果对它们进行调整，确保位置信息与分词后的文本相匹配。

# 数据预处理: 
这段代码通过两个步骤对原始数据进行了预处理，有效地将文本数据转换为了模型可以直接使用的特征，满足了项目对数据预处理的基本要求。
* 灵活性和扩展性: 代码结构清晰，易于理解和修改。为本次实验要求的进一步探索要求应用新模型，只需对相应的部分进行小的调整即可。
* 效率: 代码使用了gzip和pickle对处理后的数据进行压缩和序列化，提高了存储效率。同时，使用BertTokenizer进行高效的文本分词，并利用tqdm展示处理进度，使得我能够清晰的看到任务进度。

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import json
import gzip
import pickle
from tqdm import tqdm
from transformers import BertTokenizer


class Example(object):
    # 初始化Example类，包含问题和文档的相关信息
    def __init__(self,
                 qas_id,
                 qas_type,
                 doc_tokens,
                 question_text,
                 sent_num,
                 sent_names,
                 sup_fact_id,
                 para_start_end_position,
                 sent_start_end_position,
                 entity_start_end_position,
                 orig_answer_text=None,
                 start_position=None,
                 end_position=None):
        self.qas_id = qas_id
        self.qas_type = qas_type
        self.doc_tokens = doc_tokens
        self.question_text = question_text
        self.sent_num = sent_num
        self.sent_names = sent_names
        self.sup_fact_id = sup_fact_id
        self.para_start_end_position = para_start_end_position
        self.sent_start_end_position = sent_start_end_position
        self.entity_start_end_position = entity_start_end_position
        self.orig_answer_text = orig_answer_text
        self.start_position = start_position
        self.end_position = end_position


class InputFeatures(object):
    """数据的一组特征。"""

    def __init__(self,
                 qas_id,
                 doc_tokens,
                 doc_input_ids,
                 doc_input_mask,
                 doc_segment_ids,
                 query_tokens,
                 query_input_ids,
                 query_input_mask,
                 query_segment_ids,
                 sent_spans,
                 sup_fact_ids,
                 ans_type,
                 token_to_orig_map,
                 start_position=None,
                 end_position=None):

        self.qas_id = qas_id
        self.doc_tokens = doc_tokens
        self.doc_input_ids = doc_input_ids
        self.doc_input_mask = doc_input_mask
        self.doc_segment_ids = doc_segment_ids

        self.query_tokens = query_tokens
        self.query_input_ids = query_input_ids
        self.query_input_mask = query_input_mask
        self.query_segment_ids = query_segment_ids

        self.sent_spans = sent_spans
        self.sup_fact_ids = sup_fact_ids
        self.ans_type = ans_type
        self.token_to_orig_map = token_to_orig_map

        self.start_position = start_position
        self.end_position = end_position


def check_in_full_paras(answer, paras):
    # 检查答案是否在所有段落中
    full_doc = ""
    for p in paras:
        full_doc += " ".join(p[1])
    return answer in full_doc


def read_examples(full_file):
    # 读取示例数据
    with open(full_file, 'r', encoding='utf-8') as reader:
        full_data = json.load(reader)

    def is_whitespace(c):
        # 判断字符是否为空白字符
        if c == " " or c == "\t" or c == "\r" or c == "\n" or ord(c) == 0x202F:
            return True
        return False

    cnt = 0
    examples = []
    for case in tqdm(full_data):
        key = case['_id']
        qas_type = ""  # case['type']
        sup_facts = set([(sp[0], sp[1]) for sp in case['supporting_facts']])
        sup_titles = set([sp[0] for sp in case['supporting_facts']])
        orig_answer_text = case['answer']

        sent_id = 0
        doc_tokens = []
        sent_names = []
        sup_facts_sent_id = []
        sent_start_end_position = []
        para_start_end_position = []
        entity_start_end_position = []
        ans_start_position, ans_end_position = [], []

        # 判断答案类型
        JUDGE_FLAG = orig_answer_text == 'yes' or orig_answer_text == 'no' or orig_answer_text == 'unknown' or orig_answer_text == ""
        FIND_FLAG = False

        char_to_word_offset = []  # 累积所有句子的字符到单词的偏移
        prev_is_whitespace = True

        titles = set()
        para_data = case['context']
        for paragraph in para_data:
            title = paragraph[0]
            sents = paragraph[1]

            titles.add(title)
            is_gold_para = 1 if title in sup_titles else 0

            para_start_position = len(doc_tokens)

            for local_sent_id, sent in enumerate(sents):
                if local_sent_id >= 100:
                    break

                # 确定支持事实的全局句子ID
                local_sent_name = (title, local_sent_id)
                sent_names.append(local_sent_name)
                if local_sent_name in sup_facts:
                    sup_facts_sent_id.append(sent_id)
                sent_id += 1
                sent = " ".join(sent)
                sent += " "

                sent_start_word_id = len(doc_tokens)
                sent_start_char_id = len(char_to_word_offset)

                for c in sent:
                    if is_whitespace(c):
                        prev_is_whitespace = True
                    else:
                        if prev_is_whitespace:
                            doc_tokens.append(c)
                        else:
                            doc_tokens[-1] += c
                        prev_is_whitespace = False
                    char_to_word_offset.append(len(doc_tokens) - 1)

                sent_end_word_id = len(doc_tokens) - 1
                sent_start_end_position.append((sent_start_word_id, sent_end_word_id))

                # 答案字符位置
                answer_offsets = []
                offset = -1

                tmp_answer = " ".join(orig_answer_text)
                while True:
                    offset = sent.find(tmp_answer, offset + 1)
                    if offset != -1:
                        answer_offsets.append(offset)
                    else:
                        break

                if not JUDGE_FLAG and not FIND_FLAG and len(answer_offsets) > 0:
                    FIND_FLAG = True
                    for answer_offset in answer_offsets:
                        start_char_position = sent_start_char_id + answer_offset
                        end_char_position = start_char_position + len(tmp_answer) - 1

                        ans_start_position.append(char_to_word_offset[start_char_position])
                        ans_end_position.append(char_to_word_offset[end_char_position])

                if len(doc_tokens) > 382:
                    break
            para_end_position = len(doc_tokens) - 1

            para_start_end_position.append((para_start_position, para_end_position, title, is_gold_para))

        if len(ans_end_position) > 1:
            cnt += 1
        if key < 10:
            print("qid {}".format(key))
            print("qas type {}".format(qas_type))
            print("doc tokens {}".format(doc_tokens))
            print("question {}".format(case['question']))
            print("sent num {}".format(sent_id + 1))
            print("sup face id {}".format(sup_facts_sent_id))
            print("para_start_end_position {}".format(para_start_end_position))
            print("sent_start_end_position {}".format(sent_start_end_position))
            print("entity_start_end_position {}".format(entity_start_end_position))
            print("orig_answer_text {}".format(orig_answer_text))
            print("ans_start_position {}".format(ans_start_position))
            print("ans_end_position {}".format(ans_end_position))

        example = Example(
            qas_id=key,
            qas_type=qas_type,
            doc_tokens=doc_tokens,
            question_text=case['question'],
            sent_num=sent_id + 1,
            sent_names=sent_names,
            sup_fact_id=sup_facts_sent_id,
            para_start_end_position=para_start_end_position,
            sent_start_end_position=sent_start_end_position,
            entity_start_end_position=entity_start_end_position,
            orig_answer_text=orig_answer_text,
            start_position=ans_start_position,
            end_position=ans_end_position)
        examples.append(example)
    return examples


def convert_examples_to_features(examples, tokenizer, max_seq_length, max_query_length):
    # 将示例转换为特征
    features = []
    failed = 0
    for (example_index, example) in enumerate(tqdm(examples)):
        if example.orig_answer_text == 'yes':
            ans_type = 1
        elif example.orig_answer_text == 'no':
            ans_type = 2
        elif example.orig_answer_text == 'unknown':
            ans_type = 3
        else:
            ans_type = 0  # 统计答案类型

        query_tokens = ["[CLS]"]
        for token in example.question_text.split(' '):
            query_tokens.extend(tokenizer.tokenize(token))
        if len(query_tokens) > max_query_length - 1:
            query_tokens = query_tokens[:max_query_length - 1]
        query_tokens.append("[SEP]")

        sentence_spans = []
        all_doc_tokens = []
        orig_to_tok_index = []
        orig_to_tok_back_index = []
        tok_to_orig_index = [0] * len(query_tokens)

        all_doc_tokens = ["[CLS]"]
        for token in example.question_text.split(' '):
            all_doc_tokens.extend(tokenizer.tokenize(token))
        if len(all_doc_tokens) > max_query_length - 1:
            all_doc_tokens = all_doc_tokens[:max_query_length - 1]
        all_doc_tokens.append("[SEP]")

        for (i, token) in enumerate(example.doc_tokens):
            orig_to_tok_index.append(len(all_doc_tokens))
            sub_tokens = tokenizer.tokenize(token)
            for sub_token in sub_tokens:
                tok_to_orig_index.append(i)
                all_doc_tokens.append(sub_token)
            orig_to_tok_back_index.append(len(all_doc_tokens) - 1)

        def relocate_tok_span(orig_start_position, orig_end_position, orig_text):
            # 重新定位token化后的答案位置
            if orig_start_position is None:
                return 0, 0

            tok_start_position = orig_to_tok_index[orig_start_position]
            if orig_end_position < len(example.doc_tokens) - 1:
                tok_end_position = orig_to_tok_index[orig_end_position + 1] - 1
            else:
                tok_end_position = len(all_doc_tokens) - 1

            return _improve_answer_span(
                all_doc_tokens, tok_start_position, tok_end_position, tokenizer, orig_text)

        ans_start_position, ans_end_position = [], []
        for ans_start_pos, ans_end_pos in zip(example.start_position, example.end_position):
            s_pos, e_pos = relocate_tok_span(ans_start_pos, ans_end_pos, example.orig_answer_text)
            ans_start_position.append(s_pos)
            ans_end_position.append(e_pos)

        for sent_span in example.sent_start_end_position:
            if sent_span[0] >= len(orig_to_tok_index) or sent_span[0] >= sent_span[1]:
                continue
            sent_start_position = orig_to_tok_index[sent_span[0]]
            sent_end_position = orig_to_tok_back_index[sent_span[1]]
            sentence_spans.append((sent_start_position, sent_end_position))

        all_doc_tokens = all_doc_tokens[:max_seq_length - 1] + ["[SEP]"]
        doc_input_ids = tokenizer.convert_tokens_to_ids(all_doc_tokens)
        query_input_ids = tokenizer.convert_tokens_to_ids(query_tokens)

        doc_input_mask = [1] * len(doc_input_ids)
        doc_segment_ids = [0] * len(query_input_ids) + [1] * (len(doc_input_ids) - len(query_input_ids))

        while len(doc_input_ids) < max_seq_length:
            doc_input_ids.append(0)
            doc_input_mask.append(0)
            doc_segment_ids.append(0)

        query_input_mask = [1] * len(query_input_ids)
        query_segment_ids = [0] * len(query_input_ids)

        while len(query_input_ids) < max_query_length:
            query_input_ids.append(0)
            query_input_mask.append(0)
            query_segment_ids.append(0)

        assert len(doc_input_ids) == max_seq_length
        assert len(doc_input_mask) == max_seq_length
        assert len(doc_segment_ids) == max_seq_length
        assert len(query_input_ids) == max_query_length
        assert len(query_input_mask) == max_query_length
        assert len(query_segment_ids) == max_query_length

        sentence_spans = get_valid_spans(sentence_spans, max_seq_length)

        sup_fact_ids = example.sup_fact_id
        sent_num = len(sentence_spans)
        sup_fact_ids = [sent_id for sent_id in sup_fact_ids if sent_id < sent_num]
        if len(sup_fact_ids) != len(example.sup_fact_id):
            failed += 1
        if example.qas_id < 10:
            print("qid {}".format(example.qas_id))
            print("all_doc_tokens {}".format(all_doc_tokens))
            print("doc_input_ids {}".format(doc_input_ids))
            print("doc_input_mask {}".format(doc_input_mask))
            print("doc_segment_ids {}".format(doc_segment_ids))
            print("query_tokens {}".format(query_tokens))
            print("query_input_ids {}".format(query_input_ids))
            print("query_input_mask {}".format(query_input_mask))
            print("query_segment_ids {}".format(query_segment_ids))
            print("sentence_spans {}".format(sentence_spans))
            print("sup_fact_ids {}".format(sup_fact_ids))
            print("ans_type {}".format(ans_type))
            print("tok_to_orig_index {}".format(tok_to_orig_index))
            print("ans_start_position {}".format(ans_start_position))
            print("ans_end_position {}".format(ans_end_position))

        features.append(
            InputFeatures(qas_id=example.qas_id,
                          doc_tokens=all_doc_tokens,
                          doc_input_ids=doc_input_ids,
                          doc_input_mask=doc_input_mask,
                          doc_segment_ids=doc_segment_ids,
                          query_tokens=query_tokens,
                          query_input_ids=query_input_ids,
                          query_input_mask=query_input_mask,
                          query_segment_ids=query_segment_ids,
                          sent_spans=sentence_spans,
                          sup_fact_ids=sup_fact_ids,
                          ans_type=ans_type,
                          token_to_orig_map=tok_to_orig_index,
                          start_position=ans_start_position,
                          end_position=ans_end_position)
        )
    return features


def _largest_valid_index(spans, limit):
    # 获取最大的有效索引
    for idx in range(len(spans)):
        if spans[idx][1] >= limit:
            return idx


def get_valid_spans(spans, limit):
    # 获取有效的跨度
    new_spans = []
    for span in spans:
        if span[1] < limit:
            new_spans.append(span)
        else:
            new_span = list(span)
            new_span[1] = limit - 1
            new_spans.append(tuple(new_span))
            break
    return new_spans


def _improve_answer_span(doc_tokens, input_start, input_end, tokenizer,
                         orig_answer_text):
    """返回更好匹配标注答案的token化答案跨度。"""

    tok_answer_text = " ".join(tokenizer.tokenize(orig_answer_text))

    for new_start in range(input_start, input_end + 1):
        for new_end in range(input_end, new_start - 1, -1):
            text_span = " ".join(doc_tokens[new_start:(new_end + 1)])
            if text_span == tok_answer_text:
                return new_start, new_end

    return input_start, input_end


# 主函数示例
if __name__ == '__main__':
    parser = argparse.ArgumentParser()

    parser.add_argument("--full_data", type=str, required=True, help="Path to the full data JSON file.")
    parser.add_argument("--tokenizer_path", type=str, required=True, help="Path to pre-trained tokenizer")
    parser.add_argument("--example_output", required=True, type=str, help="Path for the processed examples")
    parser.add_argument("--feature_output", required=True, type=str, help="Path for the converted features")
    parser.add_argument("--max_seq_length", default=512, type=int, help="Maximum sequence length of the input.")
    parser.add_argument("--batch_size", default=15, type=int, help="Batch size for predictions.")
    parser.add_argument("--do_lower_case", default=True, action='store_true', help="Lower case the input text. Should be True for uncased models.")

    args = parser.parse_args()

    # 加载分词器
    tokenizer = BertTokenizer.from_pretrained(args.tokenizer_path, do_lower_case=args.do_lower_case)

    # 读取数据并转换为内部示例格式
    examples = read_examples(args.full_data)
    with gzip.open(args.example_output, 'wb') as fout:
        pickle.dump(examples, fout)

    # 将示例数据转为模型可以处理的特征
    features = convert_examples_to_features(examples, tokenizer, max_seq_length=args.max_seq_length, max_query_length=50)
    with gzip.open(args.feature_output, 'wb') as fout:
        pickle.dump(features, fout)

# run_cail
涵盖了从数据加载、模型初始化、训练、评估到结果保存的整个过程。

* 主要组成部分
* 参数和配置设置：通过set_config函数从config.py文件中导入训练和模型的配置参数。
* 数据处理：使用DataHelper类处理数据，包括数据加载和预处理，将原始数据转换为模型可直接使用的格式。
* 模型初始化：加载预训练的BERT模型，并基于此初始化项目特定的模型（BertSupportNet），同时设置优化器和学习率调度器。
* 训练函数：定义了train_epoch和train_batch函数，用于控制模型的训练过程，包括数据批处理、损失计算、反向传播等。
* 评估函数：定义了predict函数，用于在模型训练过程中或训练完成后进行模型评估，生成答案和支持性事实（sp）的预测结果，并保存到文件。
* 损失函数：定义了计算损失的函数，考虑了答案位置预测、答案类型预测和支持性事实预测的损失。
* 随机种子设置：为了实验的可复现性，提供了set_seed函数来设置随机种子。

In [ ]:
import argparse
from os.path import join
from tqdm import tqdm
from transformers import BertModel
from transformers import BertConfig as BC

from transformers.optimization import AdamW, get_linear_schedule_with_warmup
from model.modeling import *
from tools.utils import convert_to_tokens
from tools.data_iterator_pack import IGNORE_INDEX
import numpy as np
import queue
import random
from config import set_config
from tools.data_helper import DataHelper
from data_process import InputFeatures, Example
try:
    from apex import amp
except Exception:
    print('Apex not imported!')

import torch
from torch import nn


def set_seed(args):
    # 设置随机种子以确保结果的可重复性
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)


def dispatch(context_encoding, context_mask, batch, device):
    # 将context_encoding和context_mask移动到指定的GPU设备上
    batch['context_encoding'] = context_encoding.cuda(device)
    batch['context_mask'] = context_mask.float().cuda(device)
    return batch


def compute_loss(batch, start_logits, end_logits, type_logits, sp_logits, start_position, end_position):
    # 计算损失函数，包括答案的起始和结束位置损失、问题类型损失和支持事实损失
    loss1 = criterion(start_logits, batch['y1']) + criterion(end_logits, batch['y2'])
    loss2 = args.type_lambda * criterion(type_logits, batch['q_type'])

    sent_num_in_batch = batch["start_mapping"].sum()
    loss3 = args.sp_lambda * sp_loss_fct(sp_logits.view(-1), batch['is_support'].float().view(-1)).sum() / sent_num_in_batch
    loss = loss1 + loss2 + loss3
    return loss, loss1, loss2, loss3


import json

@torch.no_grad()
def predict(model, dataloader, example_dict, feature_dict, prediction_file, need_sp_logit_file=False):
    # 预测函数，不计算梯度
    model.eval()
    answer_dict = {}
    sp_dict = {}
    dataloader.refresh()
    total_test_loss = [0] * 5

    for batch in tqdm(dataloader):
        batch['context_mask'] = batch['context_mask'].float()
        start_logits, end_logits, type_logits, sp_logits, start_position, end_position = model(batch)

        loss_list = compute_loss(batch, start_logits, end_logits, type_logits, sp_logits, start_position, end_position)

        for i, l in enumerate(loss_list):
            if not isinstance(l, int):
                total_test_loss[i] += l.item()

        # 将预测结果转换为答案
        answer_dict_ = convert_to_tokens(example_dict, feature_dict, batch['ids'], start_position.data.cpu().numpy().tolist(),
                                         end_position.data.cpu().numpy().tolist(), np.argmax(type_logits.data.cpu().numpy(), 1))
        answer_dict.update(answer_dict_)

        # 预测支持事实
        predict_support_np = torch.sigmoid(sp_logits).data.cpu().numpy()
        for i in range(predict_support_np.shape[0]):
            cur_sp_pred = []
            cur_id = batch['ids'][i]

            cur_sp_logit_pred = []  # 用于支持事实logit输出
            for j in range(predict_support_np.shape[1]):
                if j >= len(example_dict[cur_id].sent_names):
                    break
                if need_sp_logit_file:
                    temp_title, temp_id = example_dict[cur_id].sent_names[j]
                    cur_sp_logit_pred.append((temp_title, temp_id, predict_support_np[i, j]))
                if predict_support_np[i, j] > args.sp_threshold:
                    cur_sp_pred.append(example_dict[cur_id].sent_names[j])
            sp_dict.update({cur_id: cur_sp_pred})

    new_answer_dict = {}
    for key, value in answer_dict.items():
        new_answer_dict[key] = value.replace(" ", "")
    prediction = {'answer': new_answer_dict, 'sp': sp_dict}
    with open(prediction_file, 'w', encoding='utf8') as f:
        json.dump(prediction, f, indent=4, ensure_ascii=False)

    for i, l in enumerate(total_test_loss):
        print("Test Loss{}: {}".format(i, l / len(dataloader)))
    test_loss_record.append(sum(total_test_loss[:3]) / len(dataloader))


def train_epoch(data_loader, model, predict_during_train=False):
    # 训练一个epoch
    model.train()
    pbar = tqdm(total=len(data_loader))
    epoch_len = len(data_loader)
    step_count = 0
    predict_step = epoch_len // 5
    while not data_loader.empty():
        step_count += 1
        batch = next(iter(data_loader))
        batch['context_mask'] = batch['context_mask'].float()
        train_batch(model, batch)
        del batch
        if predict_during_train and (step_count % predict_step == 0):
            predict(model, eval_dataset, dev_example_dict, dev_feature_dict,
                    join(args.prediction_path, 'pred_seed_{}_epoch_{}_{}.json'.format(args.seed, epc, step_count)))
            model_to_save = model.module if hasattr(model, 'module') else model
            torch.save(model_to_save.state_dict(), join(args.checkpoint_path, "ckpt_seed_{}_epoch_{}_{}.pth".format(args.seed, epc, step_count)))
            model.train()
        pbar.update(1)

    predict(model, eval_dataset, dev_example_dict, dev_feature_dict,
            join(args.prediction_path, 'pred_seed_{}_epoch_{}_99999.json'.format(args.seed, epc)))
    model_to_save = model.module if hasattr(model, 'module') else model
    torch.save(model_to_save.state_dict(), join(args.checkpoint_path, "ckpt_seed_{}_epoch_{}_99999.pth".format(args.seed, epc)))


def train_batch(model, batch):
    # 训练一个batch
    global global_step, total_train_loss

    start_logits, end_logits, type_logits, sp_logits, start_position, end_position = model(batch)
    loss_list = compute_loss(batch, start_logits, end_logits, type_logits, sp_logits, start_position, end_position)
    loss_list = list(loss_list)
    if args.gradient_accumulation_steps > 1:
        loss_list[0] = loss_list[0] / args.gradient_accumulation_steps
    
    if args.fp16:
        with amp.scale_loss(loss_list[0], optimizer) as scaled_loss:
            scaled_loss.backward()
    else:
        loss_list[0].backward()

    if (global_step + 1) % args.gradient_accumulation_steps == 0:
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

    global_step += 1

    for i, l in enumerate(loss_list):
        if not isinstance(l, int):
            total_train_loss[i] += l.item()

    if global_step % VERBOSE_STEP == 0:
        print("{} -- In Epoch{}: ".format(args.name, epc))
        for i, l in enumerate(total_train_loss):
            print("Avg-LOSS{}/batch/step: {}".format(i, l / VERBOSE_STEP))
        total_train_loss = [0] * 5


if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    args = set_config()

    args.n_gpu = torch.cuda.device_count()

    if args.seed == 0:
        args.seed = random.randint(0, 100)
    set_seed(args)

    helper = DataHelper(gz=True, config=args)
    args.n_type = helper.n_type  # 2

    # 设置数据集
    Full_Loader = helper.train_loader
    dev_example_dict = helper.dev_example_dict
    dev_feature_dict = helper.dev_feature_dict
    eval_dataset = helper.dev_loader

    roberta_config = BC.from_pretrained(args.bert_model)
    encoder = BertModel.from_pretrained(args.bert_model)
    args.input_dim = roberta_config.hidden_size
    model = BertSupportNet(config=args, encoder=encoder)
    if args.trained_weight is not None:
        model.load_state_dict(torch.load(args.trained_weight))
    model.to('cuda')

    # 初始化优化器和损失函数
    lr = args.lr
    t_total = len(Full_Loader) * args.epochs // args.gradient_accumulation_steps
    warmup_steps = 0.1 * t_total
    optimizer = AdamW(model.parameters(), lr=lr, eps=1e-8)
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=warmup_steps,
                                                num_training_steps=t_total)
    criterion = nn.CrossEntropyLoss(reduction='mean', ignore_index=IGNORE_INDEX)  # 交叉熵损失
    binary_criterion = nn.BCEWithLogitsLoss(reduction='mean')  # 二分类损失
    sp_loss_fct = nn.BCEWithLogitsLoss(reduction='none')  # 支持事实损失

    if args.fp16:
        import apex
        apex.amp.register_half_function(torch, "einsum")
        model, optimizer = amp.initialize(model, optimizer, opt_level='O1')

    model = torch.nn.DataParallel(model)
    model.train()

    # 训练
    global_step = epc = 0
    total_train_loss = [0] * 5
    test_loss_record = []
    VERBOSE_STEP = args.verbose_step
    while True:
        if epc == args.epochs:  # 达到设定的训练轮数后退出
            exit(0)
        epc += 1

        Loader = Full_Loader
        Loader.refresh()

        if epc > 2:
            # 在训练过程中进行预测
            train_epoch(Loader, model, predict_during_train=True)
        else:
            train_epoch(Loader, model)

# modeling
## SimplePredictionLayer 类
* 功能：此类负责生成用于预测答案开始位置、结束位置、类型以及支持性事实（supporting facts）的逻辑回归（logits）。
* 原理：它通过线性层（nn.Linear）对输入的特征进行变换，生成每个预测目标的逻辑回归。此外，它还使用了一个掩码（mask）机制来限制答案的可能位置，避免生成无效的答案区间。
## BertSupportNet 类
* 功能：该类是模型的主体，它集成了BERT模型（作为编码器）和SupportNet网络。
* 原理：通过将输入文本传递给BERT编码器获取上下文编码，然后将这些编码和其他必要信息传递给SupportNet以生成最终的预测。
## SupportNet 类
* 功能：用于处理经过BERT编码后的上下文编码，并通过SimplePredictionLayer生成预测。
* 原理：这一层主要是对SimplePredictionLayer的封装，以适应模型的整体架构。

In [ ]:
import torch
from torch import nn
from torch.autograd import Variable

import numpy as np
class SimplePredictionLayer(nn.Module):
    
    def __init__(self, config):
        super(SimplePredictionLayer, self).__init__()
        self.input_dim = config.input_dim

        self.sp_linear = nn.Linear(self.input_dim, 1)
        self.start_linear = nn.Linear(self.input_dim, 1)
        self.end_linear = nn.Linear(self.input_dim, 1)

        self.type_linear = nn.Linear(self.input_dim, config.label_type_num)   # yes/no/ans

        self.cache_S = 0
        self.cache_mask = None

    def get_output_mask(self, outer):
        # (batch, 512, 512)
        S = outer.size(1)
        if S <= self.cache_S:
            return Variable(self.cache_mask[:S, :S], requires_grad=False)
        self.cache_S = S
        # triu 生成上三角矩阵，tril生成下三角矩阵，这个相当于生成了(512, 512)的矩阵表示开始-结束的位置，答案长度最长为15
        np_mask = np.tril(np.triu(np.ones((S, S)), 0), 15)
        self.cache_mask = outer.data.new(S, S).copy_(torch.from_numpy(np_mask))
        return Variable(self.cache_mask, requires_grad=False)

    def forward(self, batch, input_state):
        query_mapping = batch['query_mapping']  # (batch, 512) 不一定是512，可能略小
        context_mask = batch['context_mask']  # bert里实际有输入的位置
        all_mapping = batch['all_mapping']  # (batch_size, 512, max_sent) 每个句子的token对应为1


        start_logits = self.start_linear(input_state).squeeze(2) - 1e30 * (1 - context_mask)
        end_logits = self.end_linear(input_state).squeeze(2) - 1e30 * (1 - context_mask)

        sp_state = all_mapping.unsqueeze(3) * input_state.unsqueeze(2)  # N x sent x 512 x 300

        sp_state = sp_state.max(1)[0]

        sp_logits = self.sp_linear(sp_state)

        type_state = torch.max(input_state, dim=1)[0]
        type_logits = self.type_linear(type_state)

        # 找结束位置用的开始和结束位置概率之和
        # (batch, 512, 1) + (batch, 1, 512) -> (512, 512)
        outer = start_logits[:, :, None] + end_logits[:, None]
        outer_mask = self.get_output_mask(outer)
        outer = outer - 1e30 * (1 - outer_mask[None].expand_as(outer))
        if query_mapping is not None:   # 这个是query_mapping (batch, 512)
            outer = outer - 1e30 * query_mapping[:, :, None]    # 不允许预测query的内容

        # 这两句相当于找到了outer中最大值的i和j坐标
        start_position = outer.max(dim=2)[0].max(dim=1)[1]
        end_position = outer.max(dim=1)[0].max(dim=1)[1]

        return start_logits, end_logits, type_logits, sp_logits.squeeze(2), start_position, end_position
class BertSupportNet(nn.Module):
    """
    joint train bert and graph fusion net
    """

    def __init__(self, config, encoder):
        super(BertSupportNet, self).__init__()
        # self.bert_model = BertModel.from_pretrained(config.bert_model)
        self.encoder = encoder
        self.graph_fusion_net = SupportNet(config)

    def forward(self, batch, debug=False):
        doc_ids, doc_mask, segment_ids = batch['context_idxs'], batch['context_mask'], batch['segment_idxs']
        # roberta不可以输入token_type_ids
        all_doc_encoder_layers = self.encoder(input_ids=doc_ids,
                                              token_type_ids=segment_ids,#可以注释
                                              attention_mask=doc_mask)[0]
        batch['context_encoding'] = all_doc_encoder_layers

        return self.graph_fusion_net(batch)


class SupportNet(nn.Module):
    """
    Packing Query Version
    """

    def __init__(self, config):
        super(SupportNet, self).__init__()
        self.config = config  # 就是args
        # self.n_layers = config.n_layers  # 2
        self.max_query_length = 50
        self.prediction_layer = SimplePredictionLayer(config)

    def forward(self, batch, debug=False):
        context_encoding = batch['context_encoding']
        predictions = self.prediction_layer(batch, context_encoding)

        start_logits, end_logits, type_logits, sp_logits, start_position, end_position = predictions

        return start_logits, end_logits, type_logits, sp_logits, start_position, end_position


# GFN
此段代码定义了一个基于BERT的问答系统模型，包括两个主要部分：

### SimplePredictionLayer
一个简单的预测层，用于从输入状态中预测答案的起始位置、结束位置、问题类型以及支持事实。主要步骤包括：

1. 定义线性层用于不同任务的预测。
2. 计算起始和结束位置的logits，并应用掩码。
3. 计算支持事实的状态和logits。
4. 计算问题类型的logits。
5. 根据logits确定答案的起始和结束位置。

### BertSupportNet
一个将BERT编码器和图融合网络结合的模型。主要步骤包括：

1. 使用BERT编码器对输入文档进行编码。
2. 将编码结果传递给图融合网络进行进一步处理。

### SupportNet
图融合网络，包含一个简单的预测层，用于最终的预测任务。

这些类和方法共同构成了一个完整的问答系统模型，能够处理输入的上下文和问题，并预测答案的相关信息。


In [ ]:
import torch
from torch import nn
from torch.autograd import Variable

import numpy as np

class SimplePredictionLayer(nn.Module):
    def __init__(self, config):
        super(SimplePredictionLayer, self).__init__()
        self.input_dim = config.input_dim

        # 定义线性层，用于预测起始位置、结束位置和支持事实
        self.sp_linear = nn.Linear(self.input_dim, 1)
        self.start_linear = nn.Linear(self.input_dim, 1)
        self.end_linear = nn.Linear(self.input_dim, 1)

        # 定义线性层，用于预测问题类型
        self.type_linear = nn.Linear(self.input_dim, config.label_type_num)

        self.cache_S = 0
        self.cache_mask = None

    def get_output_mask(self, outer):
        # 获取输出掩码，确保只考虑合理的起始和结束位置
        S = outer.size(1)
        if S <= self.cache_S:
            return Variable(self.cache_mask[:S, :S], requires_grad=False)
        self.cache_S = S

        np_mask = np.tril(np.triu(np.ones((S, S)), 0), 15)
        self.cache_mask = outer.data.new(S, S).copy_(torch.from_numpy(np_mask))
        return Variable(self.cache_mask, requires_grad=False)

    def forward(self, batch, input_state):
        query_mapping = batch['query_mapping']  # 查询映射
        context_mask = batch['context_mask']  # 上下文掩码
        all_mapping = batch['all_mapping']  # 全部映射

        # 计算起始和结束位置的logits，并应用掩码
        start_logits = self.start_linear(input_state).squeeze(2) - 1e30 * (1 - context_mask)
        end_logits = self.end_linear(input_state).squeeze(2) - 1e30 * (1 - context_mask)

        # 计算支持事实的状态
        sp_state = all_mapping.unsqueeze(3) * input_state.unsqueeze(2)
        sp_state = sp_state.max(1)[0]
        sp_logits = self.sp_linear(sp_state)

        # 计算问题类型的logits
        type_state = torch.max(input_state, dim=1)[0]
        type_logits = self.type_linear(type_state)

        outer = start_logits[:, :, None] + end_logits[:, None]
        outer_mask = self.get_output_mask(outer)
        outer = outer - 1e30 * (1 - outer_mask[None].expand_as(outer))
        if query_mapping is not None:
            outer = outer - 1e30 * query_mapping[:, :, None]

        # 获取起始和结束位置
        start_position = outer.max(dim=2)[0].max(dim=1)[1]
        end_position = outer.max(dim=1)[0].max(dim=1)[1]

        return start_logits, end_logits, type_logits, sp_logits.squeeze(2), start_position, end_position

class BertSupportNet(nn.Module):
    """
    joint train bert and graph fusion net
    """

    def __init__(self, config, encoder):
        super(BertSupportNet, self).__init__()
       
        self.encoder = encoder
        self.graph_fusion_net = SupportNet(config)

    def forward(self, batch, debug=False):
        # 从batch中获取输入数据
        doc_ids, doc_mask, segment_ids = batch['context_idxs'], batch['context_mask'], batch['segment_idxs']

        # 使用BERT编码器对输入进行编码
        all_doc_encoder_layers = self.encoder(input_ids=doc_ids,
                                              token_type_ids=segment_ids,
                                              attention_mask=doc_mask)[0]
        batch['context_encoding'] = all_doc_encoder_layers

        # 将编码后的结果传给图融合网络
        return self.graph_fusion_net(batch)


class SupportNet(nn.Module):
    """
    Packing Query Version
    """

    def __init__(self, config):
        super(SupportNet, self).__init__()
        self.config = config  
        
        self.max_query_length = 50
        self.prediction_layer = SimplePredictionLayer(config)

    def forward(self, batch, debug=False):
        # 获取上下文编码
        context_encoding = batch['context_encoding']
        # 使用预测层进行预测
        predictions = self.prediction_layer(batch, context_encoding)

        # 分别获取起始位置、结束位置、问题类型和支持事实的logits
        start_logits, end_logits, type_logits, sp_logits, start_position, end_position = predictions

        return start_logits, end_logits, type_logits, sp_logits, start_position, end_position

# Datahelper
### DataHelper

`DataHelper` 类是一个用于管理和加载数据的工具类，处理压缩文件（gzip格式）和序列化对象（pickle格式）。它主要包括以下几个部分：

### 初始化方法 `__init__`
- 接受参数 `gz` 和 `config`。
- 根据 `gz` 参数设置文件后缀名为 `.pkl.gz` 或 `.pkl`。
- 初始化多个数据占位符，如训练和开发的特征、示例和图数据。

### 属性
- `sent_limit`：句子长度限制，默认为100。
- `entity_limit`：实体长度限制，默认为80。
- `n_type`：类型数量，默认为2。
- `train_feature_file` 和 `dev_feature_file`：分别返回训练和开发特征文件路径。
- `train_example_file` 和 `dev_example_file`：分别返回训练和开发示例文件路径。

### 方法
- `get_feature_file(tag)`：根据标签返回特征文件路径。
- `get_example_file(tag)`：根据标签返回示例文件路径。
- `compress_pickle(pickle_file_name)`：压缩指定的pickle文件，并输出对象的简短描述。
- `__load__(file)`：根据文件类型（json或pickle）加载数据。
- `get_pickle_file(file_name)`：根据 `gz` 参数选择合适的方式打开pickle文件。
- `__get_or_load__(name, file)`：如果属性为None，则从文件中加载数据并赋值给属性。

### 特征数据
- `train_features` 和 `dev_features`：分别获取训练和开发特征数据。
- `train_feature_dict` 和 `dev_feature_dict`：分别返回训练和开发特征字典。

### 示例数据
- `train_examples` 和 `dev_examples`：分别获取训练和开发示例数据。
- `train_example_dict` 和 `dev_example_dict`：分别返回训练和开发示例字典。

### 数据加载
- `load_dev()`：加载开发特征和示例数据。
- `load_train()`：加载训练特征和示例数据。

### 数据加载器
- `dev_loader` 和 `train_loader`：分别返回开发和训练数据加载器。


In [ ]:
from os.path import join  # 导入os.path模块中的join函数，用于路径拼接
import gzip  # 导入gzip模块，用于文件压缩和解压
import pickle  # 导入pickle模块，用于对象序列化和反序列化
import json  # 导入json模块，用于处理JSON数据
from tqdm import tqdm  # 导入tqdm模块，用于显示进度条
from tools.data_iterator_pack import DataIteratorPack  # 从tools.data_iterator_pack模块导入DataIteratorPack类


class DataHelper:  # 定义一个DataHelper类
    def __init__(self, gz=True, config=None):  # 类的初始化方法，接受gz和config两个参数
        self.DataIterator = DataIteratorPack  # 将DataIteratorPack赋值给类的属性DataIterator
        self.gz = gz  # 设置是否使用gzip压缩
        self.suffix = '.pkl.gz' if gz else '.pkl'  # 根据是否压缩设置文件后缀

        self.data_dir = '/home/mw/project/data'  # 数据目录路径

        self.__train_features__ = None  # 训练特征数据占位符
        self.__dev_features__ = None  # 开发特征数据占位符

        self.__train_examples__ = None  # 训练示例数据占位符
        self.__dev_examples__ = None  # 开发示例数据占位符

        self.__train_graphs__ = None  # 训练图数据占位符
        self.__dev_graphs__ = None  # 开发图数据占位符

        self.__train_example_dict__ = None  # 训练示例字典占位符
        self.__dev_example_dict__ = None  # 开发示例字典占位符

        self.config = config  # 配置参数

    @property  # 定义只读属性sent_limit
    def sent_limit(self):   
        return 100  # 返回句子长度限制为100

    @property  # 定义只读属性entity_limit
    def entity_limit(self):
        return 80  # 返回实体长度限制为80

    @property  # 定义只读属性n_type
    def n_type(self):
        return 2  # 返回类型数目为2

    
    def get_feature_file(self, tag):  # 根据标签获取特征文件路径
        return join(self.data_dir, tag + '_feature' + self.suffix)

    def get_example_file(self, tag):  # 根据标签获取示例文件路径
        return join(self.data_dir, tag + '_example' + self.suffix)


    @property  # 定义只读属性train_feature_file
    def train_feature_file(self):
        return self.get_feature_file('train')

    @property  # 定义只读属性dev_feature_file
    def dev_feature_file(self):
        return self.get_feature_file('dev')

    @property  # 定义只读属性train_example_file
    def train_example_file(self):
        return self.get_example_file('train')

    @property  # 定义只读属性dev_example_file
    def dev_example_file(self):
        return self.get_example_file('dev')

    @staticmethod  # 定义静态方法compress_pickle
    def compress_pickle(pickle_file_name):  # 压缩pickle文件
        def abbr(obj):  # 定义内部函数abbr，生成对象的简短描述
            obj_str = str(obj)
            if len(obj_str) > 100:
                return obj_str[:20] + ' ... ' + obj_str[-20:]
            else:
                return obj_str

        def get_obj_dict(pickle_obj):  # 定义内部函数get_obj_dict，获取对象的字典表示
            if isinstance(pickle_obj, list):
                obj = pickle_obj[0]
            elif isinstance(pickle_obj, dict):
                obj = list(pickle_obj.values())[0]
            else:
                obj = pickle_obj
            if isinstance(obj, dict):
                return obj
            else:
                return obj.__dict__

        pickle_obj = pickle.load(open(pickle_file_name, 'rb'))  # 加载pickle对象

        for k, v in get_obj_dict(pickle_obj).items():  # 打印对象字典的键和值的简短描述
            print(k, abbr(v))
        with gzip.open(pickle_file_name + '.gz', 'wb') as fout:  # 压缩并保存pickle对象
            pickle.dump(pickle_obj, fout)
        pickle_obj = pickle.load(gzip.open(pickle_file_name + '.gz', 'rb'))  # 重新加载压缩后的pickle对象
        for k, v in get_obj_dict(pickle_obj).items():  # 打印重新加载后的对象字典的键和值的简短描述
            print(k, abbr(v))

    def __load__(self, file):  # 定义私有方法__load__，根据文件类型加载数据
        if file.endswith('json'):
            return json.load(open(file, 'r'))
        with self.get_pickle_file(file) as fin:
            print('loading', file)
            return pickle.load(fin)

    def get_pickle_file(self, file_name):  # 获取pickle文件对象，根据是否压缩选择打开方式
        if self.gz:
            return gzip.open(file_name, 'rb')
        else:
            return open(file_name, 'rb')

    def __get_or_load__(self, name, file):  # 定义私有方法__get_or_load__，获取或加载数据
        if getattr(self, name) is None:  # 如果属性为None，则加载数据
            with self.get_pickle_file(file) as fin:
                print('loading', file)
                setattr(self, name, pickle.load(fin))

        return getattr(self, name)  # 返回属性值

    # Features 特征数据
    @property
    def train_features(self):  # 定义只读属性train_features，获取训练特征数据
        return self.__get_or_load__('__train_features__', self.train_feature_file)

    @property
    def dev_features(self):  # 定义只读属性dev_features，获取开发特征数据
        return self.__get_or_load__('__dev_features__', self.dev_feature_file)

    # Examples 示例数据
    @property
    def train_examples(self):  # 定义只读属性train_examples，获取训练示例数据
        return self.__get_or_load__('__train_examples__', self.train_example_file)

    @property
    def dev_examples(self):  # 定义只读属性dev_examples，获取开发示例数据
        return self.__get_or_load__('__dev_examples__', self.dev_example_file)


    # Example dict 示例字典
    @property
    def train_example_dict(self):  # 定义只读属性train_example_dict，获取训练示例字典
        if self.__train_example_dict__ is None:
            self.__train_example_dict__ = {e.qas_id: e for e in self.train_examples}
        return self.__train_example_dict__

    @property
    def dev_example_dict(self):  # 定义只读属性dev_example_dict，获取开发示例字典
        if self.__dev_example_dict__ is None:
            self.__dev_example_dict__ = {e.qas_id: e for e in self.dev_examples}
        return self.__dev_example_dict__

    # Feature dict 特征字典
    @property
    def train_feature_dict(self):  # 定义只读属性train_feature_dict，获取训练特征字典
        return {e.qas_id: e for e in self.train_features}

    @property
    def dev_feature_dict(self):  # 定义只读属性dev_feature_dict，获取开发特征字典
        return {e.qas_id: e for e in self.dev_features}

    # Load 加载数据
    def load_dev(self):  # 加载开发数据
        return self.dev_features, self.dev_example_dict  #, self.dev_graphs

    def load_train(self):  # 加载训练数据
        return self.train_features, self.train_example_dict  #, self.train_graphs



    @property  # 定义只读属性dev_loader，获取开发数据加载器
    def dev_loader(self):
        return self.DataIterator(*self.load_dev(),   
                                 bsz=self.config.eval_batch_size,
                                 device='cuda:{}'.format(self.config.model_gpu),
                                 sent_limit=self.sent_limit,  # 句子长度限制为25
                                 entity_limit=self.entity_limit,
                                 sequential=True,
                                )

    @property  # 定义只读属性train_loader，获取训练数据加载器
    def train_loader(self):
        return self.DataIterator(*self.load_train(),  # example, feature, graph
                                 bsz=self.config.batch_size,
                                 device='cuda:{}'.format(self.config.model_gpu),   
                                 sent_limit=self.sent_limit,
                                 entity_limit=self.entity_limit,
                                 sequential=False
            )


# data_iterator_pack


### 初始化方法 `__init__`
- 接受参数 `features`、`example_dict`、`bsz`、`device`、`sent_limit`、`entity_limit` 和 `sequential`。
- 设置批处理大小、设备、特征列表、示例字典、句子长度限制和是否按顺序处理。
- 如果不按顺序处理，则打乱特征列表。

### 方法
- `refresh()`：重置示例指针，并在非顺序处理的情况下重新打乱特征列表。
- `empty()`：检查是否已处理完所有特征。
- `__len__()`：返回迭代器的总批次数量。
- `__iter__()`：迭代特征列表，生成批次数据并设置相关的输入和标签张量。

### 数据生成过程
- 初始化BERT输入张量和标签张量。
- 在迭代过程中，根据批次大小和当前处理的位置生成当前批次。
- 对当前批次按文档输入掩码总和排序，并设置相关映射和支持标志。
- 根据答案类型设置标签张量。
- 最后返回包含所有输入和标签的字典。

这个类的主要功能是为模型提供预处理后的批次数据，并确保数据在每次迭代时正确设置和准备。

In [ ]:

import torch
import numpy as np
from numpy.random import shuffle

IGNORE_INDEX = -100  # 忽略索引，通常用于标记无效的目标值


class DataIteratorPack(object):
    def __init__(self, features, example_dict, bsz, device, sent_limit, entity_limit,
                 entity_type_dict=None, sequential=False):
        self.bsz = bsz  # 批处理大小
        self.device = device  # 设备（如cuda或cpu）
        self.features = features  # 特征列表
        self.example_dict = example_dict  # 示例字典
        self.sequential = sequential  # 是否按顺序处理
        self.sent_limit = sent_limit  # 句子长度限制
        self.example_ptr = 0  # 当前处理的示例指针
        if not sequential:
            shuffle(self.features)  # 如果不按顺序处理，则打乱特征

    def refresh(self):
        self.example_ptr = 0  # 重置示例指针
        if not self.sequential:
            shuffle(self.features)  # 重新打乱特征

    def empty(self):
        return self.example_ptr >= len(self.features)  # 检查是否已处理完所有特征

    def __len__(self):
        return int(np.ceil(len(self.features) / self.bsz))  # 返回迭代器的总批次数量

    def __iter__(self):
        # BERT输入张量
        context_idxs = torch.LongTensor(self.bsz, 512)
        context_mask = torch.LongTensor(self.bsz, 512)
        segment_idxs = torch.LongTensor(self.bsz, 512)

        query_mapping = torch.Tensor(self.bsz, 512).cuda(self.device)
        start_mapping = torch.Tensor(self.bsz, self.sent_limit, 512).cuda(self.device)
        all_mapping = torch.Tensor(self.bsz, 512, self.sent_limit).cuda(self.device)

        # 标签张量
        y1 = torch.LongTensor(self.bsz).cuda(self.device)
        y2 = torch.LongTensor(self.bsz).cuda(self.device)
        q_type = torch.LongTensor(self.bsz).cuda(self.device)
        is_support = torch.FloatTensor(self.bsz, self.sent_limit).cuda(self.device)

        while True:
            if self.example_ptr >= len(self.features):
                break  # 如果所有特征都处理完，则跳出循环
            start_id = self.example_ptr  # 当前批处理的起始位置
            cur_bsz = min(self.bsz, len(self.features) - start_id)  # 当前批处理的大小
            cur_batch = self.features[start_id: start_id + cur_bsz]  # 当前批处理的特征
            cur_batch.sort(key=lambda x: sum(x.doc_input_mask), reverse=True)  # 按文档输入掩码总和排序

            ids = []
            max_sent_cnt = 0
            for mapping in [start_mapping, all_mapping, query_mapping]:
                mapping.zero_()  # 将映射张量置零

            is_support.fill_(0)  # 将支持标志张量置零

            for i in range(len(cur_batch)):
                case = cur_batch[i]  # 当前示例
                context_idxs[i].copy_(torch.Tensor(case.doc_input_ids))
                context_mask[i].copy_(torch.Tensor(case.doc_input_mask))
                segment_idxs[i].copy_(torch.Tensor(case.doc_segment_ids))

                for j in range(case.sent_spans[0][0] - 1):
                    query_mapping[i, j] = 1  # 设置查询映射

                # 根据答案类型设置标签
                if case.ans_type == 0:
                    if len(case.end_position) == 0:
                        y1[i] = y2[i] = 0
                    elif case.end_position[0] < 512:
                        y1[i] = case.start_position[0]
                        y2[i] = case.end_position[0]
                    else:
                        y1[i] = y2[i] = 0
                    q_type[i] = 0
                elif case.ans_type == 1:
                    y1[i] = IGNORE_INDEX
                    y2[i] = IGNORE_INDEX
                    q_type[i] = 1
                elif case.ans_type == 2:
                    y1[i] = IGNORE_INDEX
                    y2[i] = IGNORE_INDEX
                    q_type[i] = 2
                elif case.ans_type == 3:
                    y1[i] = IGNORE_INDEX
                    y2[i] = IGNORE_INDEX
                    q_type[i] = 3

                for j, sent_span in enumerate(case.sent_spans[:self.sent_limit]):
                    is_sp_flag = j in case.sup_fact_ids  # 检查是否为支持句子
                    start, end = sent_span
                    if start < end:
                        is_support[i, j] = int(is_sp_flag)  # 设置支持标志
                        all_mapping[i, start:end + 1, j] = 1  # 设置全局映射
                        start_mapping[i, j, start] = 1  # 设置起始映射

                ids.append(case.qas_id)  # 添加问题ID
                max_sent_cnt = max(max_sent_cnt, len(case.sent_spans))

            input_lengths = (context_mask[:cur_bsz] > 0).long().sum(dim=1)
            max_c_len = int(input_lengths.max())

            self.example_ptr += cur_bsz  # 更新示例指针

            yield {
                'context_idxs': context_idxs[:cur_bsz, :max_c_len].contiguous(),
                'context_mask': context_mask[:cur_bsz, :max_c_len].contiguous(),
                'segment_idxs': segment_idxs[:cur_bsz, :max_c_len].contiguous(),
                'query_mapping': query_mapping[:cur_bsz, :max_c_len].contiguous(),
                'y1': y1[:cur_bsz],
                'y2': y2[:cur_bsz],
                'ids': ids,
                'q_type': q_type[:cur_bsz],
                'start_mapping': start_mapping[:cur_bsz, :max_sent_cnt, :max_c_len],
                'all_mapping': all_mapping[:cur_bsz, :max_c_len, :max_sent_cnt],
                'is_support': is_support[:cur_bsz, :max_sent_cnt].contiguous(),
            }


### get_final_text

`get_final_text`函数将token化后的预测文本（`pred_text`）映射回原始文本（`orig_text`）。这个函数的目的是将预测的文本从经过WordPiece标记化的形式转换回原始的、更自然的文本表示。具体步骤如下：

1. 使用`BasicTokenizer`进行token化。
2. 找到`pred_text`在`orig_text`中的起始位置。
3. 如果找不到，则返回原始文本。
4. 去除空格，并创建字符对字符的映射。
5. 确保去除空格后的文本长度相同。
6. 使用字符对字符的映射将`pred_text`中的字符映射回`orig_text`。
7. 返回映射后的最终文本。

### convert_to_tokens

`convert_to_tokens`函数将模型的输出转换为可读的答案文本。它接受参数`example`、`features`、`ids`、`y1`、`y2`和`q_type`，并生成一个包含问题ID和答案文本的字典。具体步骤如下：

1. 初始化答案字典`answer_dict`。
2. 遍历每个问题ID：
   - 如果问题类型为0（文本答案），则根据预测的起始和结束位置从特征中提取对应的文档token。
   - 使用`token_to_orig_map`将token映射回原始文档的位置。
   - 将预测的token去除空格、去除WordPiece标记、清理空白，并映射回原始文本，得到最终答案文本。
   - 如果问题类型为1，则答案为“yes”。
   - 如果问题类型为2，则答案为“no”。
   - 如果问题类型为3，则答案为“unknown”。
3. 将答案文本添加到答案字典中并返回。

这两个函数的共同作用是将模型的预测结果转换为原始文本形式，并生成最终的答案文本。

In [ ]:
import collections
from transformers import BasicTokenizer
import logging

def get_final_text(pred_text, orig_text, do_lower_case, verbose_logging=False):
    """将标记化的预测文本映射回原始文本。"""

    # 当我们创建数据时，我们跟踪了原始（空格标记）tokens与我们的WordPiece标记tokens之间的对齐。
    # 因此，现在`orig_text`包含我们预测的span对应的原始文本的span。
    # 然而，`orig_text`可能包含我们不希望出现在预测中的多余字符。
    #
    # 例如，假设：
    #   pred_text = steve smith
    #   orig_text = Steve Smith's
    #
    # 我们不希望返回`orig_text`，因为它包含多余的"'s"。
    #
    # 我们也不希望返回`pred_text`，因为它已经被规范化（SQuAD评估脚本也会进行标点符号剥离/小写处理，但我们的标记器进行了额外的规范化，如去除重音字符）。
    #
    # 我们真正想返回的是"Steve Smith"。
    #
    # 因此，我们必须在`pred_text`和`orig_text`之间应用一个半复杂的对齐启发式方法，以获得字符对字符的对齐。
    # 这在某些情况下会失败，在这种情况下我们只返回`orig_text`。

    def _strip_spaces(text):
        """去除空格并建立非空字符到原始字符的映射。"""
        ns_chars = []  # 非空字符列表
        ns_to_s_map = collections.OrderedDict()  # 非空字符到原始字符的映射
        for (i, c) in enumerate(text):
            if c == " ":
                continue
            ns_to_s_map[len(ns_chars)] = i
            ns_chars.append(c)
        ns_text = "".join(ns_chars)
        return (ns_text, ns_to_s_map)

    tokenizer = BasicTokenizer(do_lower_case=do_lower_case)  # 初始化BasicTokenizer
    tok_text = " ".join(tokenizer.tokenize(orig_text))  # 标记化原始文本

    start_position = tok_text.find(pred_text)  # 找到预测文本在标记化文本中的起始位置
    if start_position == -1:  # 如果找不到
        if verbose_logging:
            print("无法在'%s'中找到文本：'%s'" % (orig_text, pred_text))
        return orig_text  # 返回原始文本
    end_position = start_position + len(pred_text) - 1  # 计算结束位置
    (orig_ns_text, orig_ns_to_s_map) = _strip_spaces(orig_text)  # 去除原始文本的空格
    (tok_ns_text, tok_ns_to_s_map) = _strip_spaces(tok_text)  # 去除标记化文本的空格

    if len(orig_ns_text) != len(tok_ns_text):  # 如果去除空格后的文本长度不同
        if verbose_logging:
            logging.info("去除空格后的长度不相等：'%s' vs '%s'", orig_ns_text, tok_ns_text)
        return orig_text  # 返回原始文本

    # 投射字符到字符的对齐
    tok_s_to_ns_map = {}
    for (i, tok_index) in tok_ns_to_s_map.items():
        tok_s_to_ns_map[tok_index] = i

    orig_start_position = None
    if start_position in tok_s_to_ns_map:
        ns_start_position = tok_s_to_ns_map[start_position]
        if ns_start_position in orig_ns_to_s_map:
            orig_start_position = orig_ns_to_s_map[ns_start_position]

    if orig_start_position is None:  # 如果起始位置无法映射
        if verbose_logging:
            print("无法映射起始位置")
        return orig_text

    orig_end_position = None
    if end_position in tok_s_to_ns_map:
        ns_end_position = tok_s_to_ns_map[end_position]
        if ns_end_position in orig_ns_to_s_map:
            orig_end_position = orig_ns_to_s_map[ns_end_position]

    if orig_end_position is None:  # 如果结束位置无法映射
        if verbose_logging:
            print("无法映射结束位置")
        return orig_text

    output_text = orig_text[orig_start_position:(orig_end_position + 1)]  # 提取最终文本
    return output_text

def convert_to_tokens(example, features, ids, y1, y2, q_type):
    """将模型的输出转换为可读的答案文本。"""
    answer_dict = dict()

    for i, qid in enumerate(ids):  # 遍历每个问题ID
        answer_text = ''
        if q_type[i] == 0:  # 文本答案
            doc_tokens = features[qid].doc_tokens  # 获取文档tokens
            tok_tokens = doc_tokens[y1[i]: y2[i] + 1]  # 提取预测的tokens
            tok_to_orig_map = features[qid].token_to_orig_map  # token到原始文本的映射
            if y2[i] < len(tok_to_orig_map):  # 如果结束位置在映射范围内
                orig_doc_start = tok_to_orig_map[y1[i]]
                orig_doc_end = tok_to_orig_map[y2[i]]
                orig_tokens = example[qid].doc_tokens[orig_doc_start:(orig_doc_end + 1)]  # 提取原始tokens
                tok_text = " ".join(tok_tokens)  # 拼接token文本

                # 去除WordPiece标记
                tok_text = tok_text.replace(" ##", "")
                tok_text = tok_text.replace("##", "")

                # 清理空白
                tok_text = tok_text.strip()
                tok_text = " ".join(tok_text.split())
                orig_text = " ".join(orig_tokens).strip('[,.;]')  # 去除多余字符

                # 获取最终文本
                final_text = get_final_text(tok_text, orig_text, do_lower_case=False, verbose_logging=False)
                answer_text = final_text
        elif q_type[i] == 1:  # 答案为“yes”
            answer_text = 'yes'
        elif q_type[i] == 2:  # 答案为“no”
            answer_text = 'no'
        elif q_type[i] == 3:  # 答案为“unknown”
            answer_text = 'unknown'
        answer_dict[qid] = answer_text  # 添加到答案字典中
    return answer_dict  # 返回答案字典
